In [1]:
import pandas as pd
import torch
import pickle
import numpy as np
from gensim.models import Word2Vec  

In [2]:
df = pd.read_csv('../data/text.csv')

In [3]:
# Загрузка списка из файла
with open('../data/tokenized_text.pkl', 'rb') as f:
    tokenized_text = pickle.load(f)

In [4]:
# буду использовать word2vec для эмбеддинга
w2v_model = Word2Vec(tokenized_text, vector_size=100, workers=4)
# Получу векторные представления слов для каждого документа и усредняю веркторы в документе. Сохраняю индексы документов, для которых не нашлось векторных представлений слов
empty_doc_indexes = []
document_vectors = []
for index, document in enumerate(tokenized_text):
    doc_vectors = [w2v_model.wv[word] for word in document if word in w2v_model.wv]

    if doc_vectors:
        avg_doc_vector = np.mean(doc_vectors, axis=0)
        document_vectors.append(avg_doc_vector)
    else:
        empty_doc_indexes.append(index)

In [5]:
# Удалю метки для документов, слова в которых не получили векторное представление
labels_list = df['label'].values.tolist()
for index in sorted(empty_doc_indexes, reverse=True):
    labels_list.pop(index)

In [6]:
# Создаю тензоры для данных и их меток
document_vectors_np = np.array(document_vectors)
document_vectors_tensor = torch.tensor(document_vectors_np)
labels_tensor = torch.tensor(labels_list)

In [7]:
torch.save(document_vectors_tensor, '../data/document_vectors_emb_tensor.pt')
torch.save(labels_tensor, '../data/labels_emb_tensor.pt')